### Finite Difference Methods
Finite Difference Methods (FDM) are one of the popular numerical methods used in computational finance. FDM are discretizations used for solving differential equations by approximating them with difference equations. It is one of the simplest and the oldest methods to solve differential equations. These techniques were applied to numerical applications as early as 1950s.

FDM are similar in approach to the (binomial) tress. However, instead of discretizing asset prices and the passage of time in a tree structure, it discretizes in a grid - with time and price steps - by calculating the value at every possible grid points.

Explicit, Implicit and Crank-Nicolson are the three popular approaches of FDM. The explicit methods are simple to implement, but it does not always converge and largely depends on the size of the time and asset step. Explicit methods are unstable when compared to other two methods. Finite Difference approach is peferred for low dimensional problem, usually upto 4 dimensions.

### Explicit FDM
Explicit Finite Difference Method is a numerical technique used to solve partial differential equations (PDEs) by approximating derivatives with finite differences. It is "explicit" in the sense that the new values in the grid are computed directly from the known values of the previous time step, without needing to solve a system of equations. In Explicit FDM, the solution at the next time step is computed explicitly from the known values at the current time step. This method is straightforward and involves updating each grid point using a formula derived from the PDE and its boundary conditions.

Note: Explicit FDM is a forward marching scheme in its numerical computation, but it's applied in a backward-looking manner for option pricing problems. The numerical scheme is forward marching (from known to unknown). The application to option pricing is backward-looking (from expiration to present).

### Differentiation Using The Grid
The Binomial method contains the diffusion - the volatility - in the tree structure whereas in FDM, the 'tree' is fixed and we change the parameters to reflect the changing diffusion. We will now define the grid by specifying the time step $δt$ and asset step $δs$ and discretize $S$ and $t$ as

$$ S=i\delta s $$
and time to maturity as 
$$ t=T-k \delta t $$
where 0 ≤ i ≤ I and 0 ≤ k ≤ K
$$V^{k}_{i} = V( i \delta S, T-k \delta t )$$
### Approximating Greeks
The greeks terms, the Black–Scholes equation can be written as
$$\Theta + \frac1{2}{\sigma^2 S^2}\Gamma + r S \Delta - rV = 0$$
Assume that we know the option value at each grid points, we can extract the derivatives of the option using Taylor series expansion.

## Approximating $Θ$

We know that the first derivative of option as,
 
$$\frac{\partial V}{\partial t} = \underset{h \, \rightarrow \, 0}{\lim}\, \frac{V(S, t + h) - V(S, t)}{h}$$

We can then approximate the time derivative from our grid using

$$\frac{\partial V}{\partial t} (S, t) \approx \frac{V_{i}^{k} - V_{i}^{k+1} }{\delta t}$$

## Approximating $Δ$

From the lecture, we know that the central difference has much lower error when compared to forward and backward differences. Accordingly, we can approximate the first derivative of option with respect to the underlying as

$$\frac{\partial V}{\partial S} (S, t) \approx \frac{V_{i+1}^{k} - V_{i-1}^{k} }{2 \delta S}$$

## Approximating $Γ$

The gamma of the option is the second derivative of option with respective to the underlying and approximating it we have,

$$\frac{\partial V^2}{\partial S^2} (S, t) \approx \frac{V_{i+1}^{k} - 2 V_{i}^{k} + V_{i-1}^{k} }{\delta S^2}$$

## Import Required Libraries

In [1]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Importing libraries
import pandas as pd
import numpy as np

# Import cufflinks
import cufflinks as cf
cf.set_config_file(offline=True, dimensions=((1000,600)))

# Set max row and columns to 300 and 100
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 100)

# Example

Suppose that we know the value of the option on the below grid points, we can then easily evaluate the greeks as follows

From the grid, we can estimate the

$$\Theta = \frac{12-13}{0.1} = -10$$
$$\Delta = \frac{15-10}{2x2} = 1.25$$
$$\Gamma = \frac{15-2x12+10}{2x2} = 0.25$$

Black Scholes Equation is a relationship between the option value and greeks. If we know the option value at the expiration, we can step back to get the values prior to it such that $V^i_{k} = V^i_{k-1} - \Theta * dt$. This approach is called Explicit Finite Difference Method because the relationship between the option values at time step 
$k$ is a simple function of the option values at time step $k−1$.

### Option Pricing Techniques
As with other option pricing techniques Explicit Finite Difference methods are used to price options using what is essentially a three step process.

Step 1: Generate the grid by specifying grid points.
Step 2: Specify the final or initial conditions.
Step 3: Use boundary conditions to calculate option values and step back down the grid to fill it.

## European Option

To price an option, we generate a finite grid of a specified asset and time steps for a given maturity. Next, we specify the initial and boundary conditions to calculate payoff when S and T equals zero. We then step back to fill the grid with newer values derived from the earlier values.

## Specify Parameters

In [2]:
# Specify the parameters for FDM 
T    = 1                              # time to maturity in years
E    = 100                            # strike price
r    = .05                            # riskfree rate
vol  = .20                            # volatility
call = True                           # call = True

NAS  = 20                             # number of asset steps 
ds   = 2*E/NAS                        # asset step size

dt   = (0.9/vol**2/NAS**2)            # time step size, for stability
NTS  = int(T/dt) + 1                  # number of time steps
dt   = T/NTS                          # time step size [Expiration as int # of time steps away]

american = False                      # american = False

### Generate Grid
Build the grid with the above input parameters

In [3]:
# Create asset steps i*ds
s = np.arange(0, (NAS+1)*ds, ds)
s
# Create time steps k*dt
# k goes from 0,1,...,10; # t goes from 1,0.9,...0
t= T-np.arange(NTS*dt,-dt,-dt)[::-1]
t
# Verify the steps size
s.shape, t.shape
# Initialize the grid with zeros
grid = np.zeros((len(s), len(t)))

# Subsume the grid points into a dataframe
# with asset price as index and time steps as columns
grid = pd.DataFrame(grid, index=s, columns=np.around(t, 3))
grid

,1.000,0.944,0.889,0.833,0.778,0.722,0.667,0.611,0.556,0.500,0.444,0.389,0.333,0.278,0.222,0.167,0.111,0.056,0.000
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
80.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Set up Payoff
Specify payoffs at expiration for calls
$$V_i^{0} = max(i\delta s - E, 0)$$

In [4]:
# Set Payoff at Expiration
flag = 1 if call else -1
grid.iloc[:,-1] = np.maximum(flag * (s - E), 0)
# Verify the grid
grid
# Store payoff for early exercise
p = grid.iloc[:, -1].copy()
p

0.0        0.0
10.0       0.0
20.0       0.0
30.0       0.0
40.0       0.0
50.0       0.0
60.0       0.0
70.0       0.0
80.0       0.0
90.0       0.0
100.0      0.0
110.0     10.0
120.0     20.0
130.0     30.0
140.0     40.0
150.0     50.0
160.0     60.0
170.0     70.0
180.0     80.0
190.0     90.0
200.0    100.0
Name: 0.0, dtype: float64

Fill the Grid
Specify boundary condition at $S = 0$

$$V_0^{k} = (1 - r \delta t) V_0^{k-1}$$

Specify boundary condition at $S = ∞$

$$V_i^{k} = 2V_{i-1}^{k} - V_{i-2}^{k}$$

In [5]:
# Fill the grid
for k in range(len(t)-2, -1, -1):  # Backward in time
    for i in range(1, len(s)-1):
        delta = (grid.iloc[i+1, k+1] - grid.iloc[i-1, k+1]) / (2*ds)
        gamma = (grid.iloc[i+1, k+1] - 2*grid.iloc[i, k+1] + grid.iloc[i-1, k+1]) / (ds**2)
        theta = (-0.5 * vol**2 * s[i]**2 * gamma) - (r*s[i]*delta) + (r*grid.iloc[i, k+1])
        
        grid.iloc[i, k] = grid.iloc[i, k+1] - dt*theta
    
    # Set boundary condition at S = 0
    grid.iloc[0, k] = grid.iloc[0, k+1] * np.exp(-r*dt)
    
    # Set boundary condition at S = infinity
    grid.iloc[-1, k] = 2*grid.iloc[-2, k] - grid.iloc[-3, k]
    
    # Check for early exercise
    if american:
        discount_factor = np.exp(-r * (T - t[k]))
        grid.iloc[:, k] = np.maximum(grid.iloc[:, k], p*discount_factor)

# round grid values to 2 decimals
grid = np.around(grid,2)
# Output the option values
grid
# Print out stock, payoff and option value 
data = {
    "Stock": s,
    "Payoff": p,
    "Option": grid.iloc[:,0]
}

option_value_2D = pd.DataFrame(data)
# print(option_value_2D.to_string(index=False))
option_value_2D                         
# Plot option value and payoff
option_value_2D[['Payoff', 'Option']].iplot(title='Payoff & Option Value')

### User Defined Function
Let's subsume above grid calculation into a function for ease of use. All we have to do is to combine the above code blocks into a single function.

In [6]:
# Create function to price options
def efdmoption(E=100, vol=0.20, r=0.05, T=1, NAS=20, call=True, american=False):

    ds   = 2* E / NAS                     # asset step size
    dt   = (0.9/vol**2/NAS**2)            # time step size, for stability
    NTS  = int(T / dt) + 1                # number of time steps
    dt   = T / NTS                        # time step size [Expiration as int # of time steps away]
    
    # Create asset steps i*ds
    s = np.arange(0, (NAS+1)*ds, ds)
    
    # Create time steps k*dt
    # k goes from 0,1,...,10; # t goes from 1,0.9,...0
    # t = np.arange(T+dt, 0, -dt)
    t= T-np.arange(NTS*dt,-dt,-dt)[::-1]
    
    # Initialize the grid with zeros
    grid = np.zeros((len(s), len(t)))
    
    # Subsume the grid points into a dataframe
    # with asset price as index and time steps as columns
    grid = pd.DataFrame(grid, index=s, columns=np.around(t, 3))
    
    # Set Payoff at Expiration
    flag = 1 if call else -1
    grid.iloc[:,-1] = np.maximum(flag * (s - E), 0)
    
    # Store payoff for early exercise
    p = grid.iloc[:, -1].copy()
    
    # Fill the grid
    for k in range(len(t)-2, -1, -1):  # Backward in time
        for i in range(1, len(s)-1):
            delta = (grid.iloc[i+1, k+1] - grid.iloc[i-1, k+1]) / (2*ds)
            gamma = (grid.iloc[i+1, k+1] - 2*grid.iloc[i, k+1] + grid.iloc[i-1, k+1]) / (ds**2)
            theta = (-0.5 * vol**2 * s[i]**2 * gamma) - (r*s[i]*delta) + (r*grid.iloc[i, k+1])
            
            grid.iloc[i, k] = grid.iloc[i, k+1] - dt*theta
        
        # Set boundary condition at S = 0
        grid.iloc[0, k] = grid.iloc[0, k+1] * np.exp(-r*dt)
        
        # Set boundary condition at S = infinity
        grid.iloc[-1, k] = 2*grid.iloc[-2, k] - grid.iloc[-3, k]
        
        # Check for early exercise
        if american:
            discount_factor = np.exp(-r * (T - t[k]))
            grid.iloc[:, k] = np.maximum(grid.iloc[:, k], p*discount_factor)
    
    # round grid values to 2 decimals
    return np.around(grid,2)
# Call the function to price options
fdm_grid = efdmoption()
fdm_grid

,1.000,0.944,0.889,0.833,0.778,0.722,0.667,0.611,0.556,0.500,0.444,0.389,0.333,0.278,0.222,0.167,0.111,0.056,0.000
0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
10.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
20.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
30.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
40.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
50.0,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
60.0,0.07,0.05,0.04,0.03,0.02,0.02,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
70.0,0.43,0.37,0.31,0.26,0.21,0.16,0.13,0.09,0.07,0.04,0.03,0.02,0.01,0.00,0.00,0.00,0.00,0.00,0.0
80.0,1.75,1.58,1.42,1.25,1.10,0.94,0.80,0.66,0.53,0.42,0.31,0.22,0.14,0.08,0.04,0.01,0.00,0.00,0.0
90.0,4.90,4.60,4.31,4.00,3.70,3.39,3.08,2.77,2.45,2.13,1.81,1.49,1.18,0.88,0.59,0.34,0.13,0.00,0.0


### Option Values
Let's now get the value of European & American call and put option for K=100, Volatility=20%, Rate=5%, T=1 and NAS=60.

In [7]:
# Output the option values
call = efdmoption(100,0.2,0.05,1,50,call=True, american=True).loc[100,1]
put = efdmoption(100,0.2,0.05,1,50,call=False, american=True).loc[100,1]

# Print the values
print(f"Call Option Value is {call:0.4f}")
print(f"Put Option Value is {put:0.4f}")

Call Option Value is 10.4200
Put Option Value is 5.9100


### Visualize the payoff

In [8]:
# Plot Option Payoff 
# Axis titles are not rendered correctly on the graph. This is a bug in cufflinks
fig = fdm_grid.iplot(kind = 'surface', title='Option values by Explicit FDM', xTitle='Spot', yTitle='Maturiy', zTitle='Option Value', asFigure=True)
fig.show()
# Save the figure - you might have to install kaleido for earlier version of plotly
# @pip install -U kaleido

# Save as portable network graphics
# fig.write_image("images/fdm_option.png")

### Bilinear Interpolation
We have generated the grid and filled it with the possible option values. However, if we have to estimate option value or its derivatives on the mesh points, how can we estimate the value at points in between? The simplest way is to do a two-dimensional interpolation method called Bilinear Interpolation.


The option value can be estimated using the values from the nearest neighbouring values. Assume V1,V2,V3 and V4 are the option values from the nearest neighbour and A1,A2,A3 and A4 are the areas of the rectangles made by the four corners and the interior points, we can approximate the option value at the interior points as

In [9]:
def bilinear_interpolation(asset_price, ttm, df):

    # Find relevant rows and columns
    col1 = df.columns[df.columns > ttm][-1]
    col2 = df.columns[df.columns <= ttm][0]
    row1 = df.index[df.index < asset_price,][-1]
    row2 = df.index[df.index >= asset_price,][0]
   
    # Define points and areas
    V = [df.loc[row1, col1], df.loc[row1, col2],
           df.loc[row2, col2], df.loc[row2, col1]]
                      
    A = [(row2 - asset_price) * (col2 - ttm),
           (row2 - asset_price) * (ttm - col1),
           (asset_price - row1) * (ttm - col1),
           (asset_price - row1) * (col2 - ttm)]
   
    # Interpolate values
    return np.sum(np.array(V)*np.array(A))/sum(np.array(A))

In [10]:
# Option value, approximated
bilinear_interpolation(115,0.3,fdm_grid)
# Verify rows and columns - Examples - Older version with axis flip
# col1 = grid.columns[grid.columns > 0.3][-1]
# col2 = grid.columns[grid.columns <= 0.3][0]
# row1 = grid.index[grid.index < 110][-1]
# row2 = grid.index[grid.index >= 110][0]

# Nearest neighbours grid points
# [row1,col1], [row1, col2], [row2, col2], [row2, col1]

# Get option values
# V = [grid.loc[row1,col1], grid.loc[row1,col2], grid.loc[row2, col2], grid.loc[row2, col1]]

# Areas of the rectangle made by four corners and interior points
# A = [(row2-105) * (col2-0.3),
#      (row2-105) * (0.3-col1),
#      (105-row1) * (0.3-col1),
#      (105-row1) * (col2-0.3)]

# Option value in mesh points, approximated
# sum(array(V)*array(A))/sum(array(A))

16.937

### Convergence Analysis
Let's now compare option pricing for various asset steps (NAS) with black scholes price.

In [12]:
# Iterate over asset steps (NAS)
nas_list = [10,20,30,40,50,60]
fdmoption = []
for i in nas_list:
    fdmoption.append(efdmoption(100,0.2,0.05,1,i).loc[100,1])
fdmoption
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
# Call black scholes class
from BlackScholesModel import getcalloption

# Instantiate black scholes object
option = getcalloption("call",100,100,1,0.05,0.2)
bsoption = np.round(option.price,2)
bsoption = bsoption.repeat(len(nas_list))

# Range of option price
bsoption
# Subsume into dataframe
df = pd.DataFrame(list(zip(bsoption,fdmoption)), columns=['BS', 'FDM'], index=nas_list)
df['dev'] = df['FDM'] - df['BS']
df['% dev'] = round(df['dev'] / df['BS'] * 100.,2)

# Output
print("BS - FDM Convergence over NAS")
df

BS - FDM Convergence over NAS


,BS,FDM,dev,% dev
10,10.45,9.51,-0.94,-9.00
20,10.45,10.26,-0.19,-1.82
30,10.45,10.37,-0.08,-0.77
40,10.45,10.40,-0.05,-0.48
50,10.45,10.42,-0.03,-0.29
60,10.45,10.43,-0.02,-0.19
